### compile GSI in Ubunut 14.04

- GSI was tried to compile in Ubuntu 12.04, but ended with failure. GSI requieres Gfortran 4.7 and above, ubunut 12.04 repostory is updated upto 4.6, so ubunut has to upgraded from 12.04 to 14.04 to have gfortran 4.7.
- Due to this Server was upghraded from Ubuntu 12.04 to 14.04 and compiled WRFV4.3.1 and WPS 4.3.1 in it. It was by Gfortran 4.8.2, Netcdf 4.1.3, and other necssary libraries such as libpng, zlib and jasper as per apt-get method.
- GSI is requiered to have OpenMPI for its parallell programming. For this the installation of OpenMPI was follwoed from [this](http://www.sysads.co.uk/2014/05/install-open-mpi-1-8-ubuntu-14-04-13-10/).
- OpenMpi was installed by following commands from from above refered page
```bash
wget https://www.open-mpi.org/software/ompi/v1.8/downloads/openmpi-1.8.1.tar.gz
tar -xvf openmpi-1.8.1.tar.gz
cd openmpi-1.8.1.tar.gz	
./configure --prefix="/home/$USER/.openmpi"
make
do make
export PATH="$PATH:/home/$USER/.openmpi/bin"
export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/home/$USER/.openmpi/lib/"
```
- to check the installation by ```mpirun```. Based on the last environment setup the openmpi will become the MPI library instead of mpich used for WRF3V compilation
- Following environments were setup for GSI configure
```bash
export WRF_DIR=/home/hoopoe/wrfchem341/test/WRFV3
export NETCDF=/home/hoopoe/wrfchem341/test/netcdf36
export LAPACK_PATH=/usr/lib
LAPACK_PATH environment was setup following [this](http://qiita.com/make_now_just/items/d6e251bbf18f4c141c1d)
```
- After above point, ```./configre``` was executed and configure.gsi was edited to change its line number 43 to consider the difference in gfortaan and its commanding problem mentioned [here](http://forum.wrfforum.com/viewtopic.php?f=5&t=6086&p=22377#p22377)
- AFter this made ```./compile > &log.compileA1``` and it ends in this error ```*** Error in `/usr/lib/gcc/x86_64-linux-gnu/4.8/f951': corrupted double-linked list: 0x0000000002b5e5e0 ***```. Before getting this error, faced minimal error related with mpif90 command missing etc, easily resolved those issues, need of expert direction for above error.
- Based on this error it is decided to use the exact same compiler and gfortran version mentioned in the user guide. For this the gfortran version 4.7 is installed by ```sudo apt-get install gfortran4.7``` and gcc version 4.4.5 was tried to install by typing ```sudo apt-get install gcc-4.4``` but it installed version 4.4.7 instead. A number of attempts such as commands ```sudo apt-get install 	gcc-4.4=4.4.5-15ubuntu1``` ends but in failure. 
- So based on the available compilers version, gcc-4.4.7 and g++-4.4.7, gfortran-4.7, the wrf will be compiled and tested for its suitablity for GSI.

#### compiling WRF-CHEM-3.4.1 with gcc-4.4.7, gfortan-4.7.3 and netcdf 3.6.3
- Compiling netcdf 3.6.3  

```fortran
*environment setup* **THIS IS WRONG, SEE BELOW**
export DIR=/home/hoopoe/wrfchem341/test
export CC=gcc-4.4
export CXX=g++-4.4
export FC=gfortran-4.7
export FCFLAGS=-m64
export F77=gfortran-4.7
export FFLAGS=-m64
*compiling*
cd /home/hoopoe/wrfchem341/test/netcdf-3.6.3
./configure --prefix=$DIR/netcdf36g44
make 
make install
make check
```
but ends in error saying 
```fortran
use typeSizes
      1
Fatal Error: Wrong module version '10' (expected '9') for file 'typesizes.mod' opened at (1)
``` 
- based on [this](http://stackoverflow.com/questions/13696820/wrong-module-version-on-compile) it might be due to compiler version mismatch. To choose in between different versions of same program, ```update-alternatives``` has to be used, refered by [this](http://stackoverflow.com/questions/7832892/how-to-change-the-default-gcc-compiler-in-ubuntu/7834049#7834049) and based on [this](http://charette.no-ip.com:81/programming/2011-12-24_GCCv47/). To set up gcc and g++, used follwoing commands
```fortran
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.4 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.4 
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 40 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8 
sudo update-alternatives --config gcc 
```
- To setup gfortran older version 4.7, used following commands, 
```bash
which gfortran
# to knwo the link of current used gfortran
ls -lh /usr/bin/gfortran
ls -lh /usr/bin/gfortran*
#to knwo the list of links for each version
sudo update-alternatives --install /usr/bin/gfortran gfortran	/usr/bin/gfortran-4.8 60
sudo update-alternatives --install /usr/bin/gfortran gfortran /usr/bin/gfortran-4.7 40
sudo update-alternatives --install /usr/bin/gfortran gfortran /usr/bin/gfortran-4.6 20
sudo update-alternatives --config gfortran
```
- Based on this required gfortran version may be selected and so the environment setup need not to have the version mentioned as above, so the environment setup is changed as follows
```bash
*environment setup* 
export DIR=/home/hoopoe/wrfchem341/test
export CC=gcc
export CXX=g++
export FC=gfortran
export FCFLAGS=-m64
export F77=gfortran
export FFLAGS=-m64
```
- With this setup also, compilation ends with same error as noted above, so it is concluded that the gfortran, gcc, and g++ has to be have same version, that if it is 4.4, all has to be same. So install gfortran4.4 by ```sudo apt-get install gfortran-4.4```, this version was added into ```update-alternatives``` by 
```bash
sudo update-alternatives --install /usr/bin/gfortran gfortran /usr/bin/gfortran-4.4 10
```
- With the above change also faced same ```Fatal Error: Wrong module version``` error, from [this](https://bugs.archlinux.org/task/23957) it is found that the error is due to OPENMPI compiled with wrong gfortran. Basically the gfortran when doing OPENMPI compilation based on [this](http://www.sysads.co.uk/2014/05/install-open-mpi-1-8-ubuntu-14-04-13-10/) is 4.8. So OPENMPI has to be compiled with correct gfortran needed by GSI, that is 4.7. So made ```update-alternatives``` configuration chosen to gfortran 4.7 and gcc and g++ in 4.4 then proceeded with openmpi compilation as per [this](http://www.sysads.co.uk/2014/05/install-open-mpi-1-8-ubuntu-14-04-13-10/).
- Them again tried with netcdf compilation, But got same error as of former. For a check, tried gcc and gfortran was made into 4.8 and compiled netcdf(NOte here openmpi is made by gfortran4.7), it make check passed without any error. Then again changed the gcc to 4.4 and gfortran to 4.7, this time netcdf make check passed without any error, strange!. Made exporting of compiled netcdf folder by 
```bash
export PATH=$DIR/netcdf36g44/bin:$PATH
export NETCDF=$DIR/netcdf36g44
```
- So went into testing of netcdf *NETCDF testing* from [here](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/compile_tutorial/tar_files/Fortran_C_NETCDF_MPI_tests.tar) downloaded the testing files and used, 

```bash
mkdir testNET
wget http://www2.mmm.ucar.edu/wrf/OnLineTutorial/compile_tutorial/tar_files/Fortran_C_NETCDF_MPI_tests.tar
tar -xf Fortran_C_NETCDF_MPI_tests.tar
cd testNET
cp ${NETCDF}/include/netcdf.inc .
gfortran -c 01_fortran+c+netcdf_f.f
gcc -c 01_fortran+c+netcdf_c.c
gfortran 01_fortran+c+netcdf_f.o 01_fortran+c		+netcdf_c.o      -L${NETCDF}/lib -lnetcdff -lnetcdf	
./a.out
```
- Based on good pass in testing, went into compile wrf with chemistry option as per [this](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.php), checked the environment setup as per [this](http://sergioibarra.blogspot.in/2012/10/wrf-chem-compilacion-en-ubuntu-1204.html), checked  ```sudo nano /etc/bash.bashrc``` for its correctness. Then went into configure and compilation but ends with commenting errors, the log files is [this](WRFGSIlog.compileA2) and configure file is [this](WRFGSIconfigure.wrfA2). 
- Based on the last good compilation configure [file](GSIconfigure.wrfA1), the configure file was edited in the line 119 to 120 as per this 
```bash	
FORMAT_FIXED    =       -ffixed-form -cpp
FORMAT_FEE     =       -ffree-form -ffree-line-length-none -cpp 
from 
FORMAT_FIXED    =       -ffixed-form 
FORMAT_FEE     =       -ffree-form -ffree-line-length-none
```         
- Then made attempt 3 in this regard by ```./clean``` and ```./configure``` then editing line 119 to 120 of configure [file](WRFGSIconfigure.wrfA3), and then doing ```./compile em_real >& log.compileA3```. This time it took 17 minutes and finished compilation without error and creation of all exe files. This is advancement in compiling WRF CHEM based on openMPI instead of MPICH.
- Next started GSI compilations, made sure the environment variable for GSI is correct
```bash			  
export WRF_DIR=/home/hoopoe/wrfchem341/test/WRFV3
export NETCDF=/home/hoopoe/wrfchem341/test/netcdf36
export LAPACK_PATH=/usr/lib    
```
- Then ```./configre``` was executed and [configure.gsi](GSIconfigure.gsiA11) was formed and then given ```./compile > &GSIlog.compileA11```. This attempt gives error of commending populated in the log file [here](GSIlog.compileA11). 
- Next attempt was made in this regard, given ```./clean -a```, then ```./configure```, edited the configure file from line number 43 

```bash					
CPP_FLAGS      =  -C -P -D_REAL8_ -DWRF -DLINUX -DPGI
to 
CPP_FLAGS      =  -P -D_REAL8_ -DWRF -DLINUX -DPGI
```
then executed compile by ```./compile > &GSIlog.compileA12```
- edited to change its line number 43 to consider the difference in gfortaan and its commanding problem mentioned [here](http://forum.wrfforum.com/viewtopic.php?f=5&t=6086&p=22377#p22377). With this compile took time and ends with error of no lmkl, indicating no intel mkl, it was mentioned in the confiugre.gsi file in the line 58 starts with ```MYLIBsys```, in the next attempt this line was eidted from 

```python
MYLIBsys       =  -L\$(LAPACK_PATH)/lib/intel64 -lmkl_intel_lp64 -lmkl_intel_thread -lmkl_core
to
MYLIBsys       =  -L\$(LAPACK_PATH) 
```
- Also made sure all the package listed [here](http://astroiisc.wikispaces.com/Lapack+Installation+in+Ubuntu)
```bash
sudo apt-get install libblas3gf
sudo apt-get install libblas-doc
sudo apt-get install libblas-dev
sudo apt-get install liblapack3gf
sudo apt-get install liblapack-doc
sudo apt-get install liblapack-dev
```
towards LAPCK_PATH was installed in the system. 
- Some of the notes related with advantages of mkl, such as on [R](http://www.r-bloggers.com/speeding-up-r-with-intels-math-kernel-library-mkl/) and it bench mark is astonishing with OPEN source LAPACK which currently trying to use.
- This ends with missing file error of ```ar: plib8b.o: No such file or directory```, but less compilation than last attempt 11, log file is [here](GSIlog.compileA13). This makes to speculate problem lies with LAPACK library compilation requirement than just apt-get. 
- LAPCK was compiled based on [this](https://gcc.gnu.org/testing/testing-lapack.html), in which the example file for compile ```make.inc.example``` used as such by renaming into ```make.inc``` and run the command ```PATH=$PATH:. export PATH; make install blaslib lapacklib tmglib testing blas_testing timing blas_timing``` but this ended in error of ```make: *** No rule to make target `make.inc'.  Stop.```
- As an another attempt related with failure of compiling LAPCK, a command line search is made for actual location of liblapack3 using ``` sudo dpkg -L liblapack3```. It showed the location as ```/usr/lib/lapack``` and so changed the LPACK_PATH into ```export LAPACK_PATH=/usr/lib/lapack``` and redone the compile, to surprise this time compile finished with out any error and created the ```gsi.exe``` executables, [this](GSIconfigure.gsiA14) and [this](GSIlog.compileA14) are configure and compile log file.